In [ ]:
In this tutorial, we will step line-by-line through implementation of a Bayesian linear regression model. Our data consists of mean single-unit firing rates from rat inferior colliculus in response to
varying depths of sinusoidal amplitude modulated auditory stimuli (see paper for details).

The first step as with any Python program is to import necessary helper analysis functions and toolboxes. Before running this step, make sure all required packages are installed as
detailed in the setup portion of this github profile.

In [ ]:
import numpy as np               #Numpy for numerical 'ala Matlab' computations
import pymc as pm                #pymc will be doing most of our heavy lifting for Bayesian calculations
import matplotlib.pyplot as plt  #This works as our plotting tool
import arviz as az               # arviz is a visualization package for plotting probabilistic variables, such as prior or posterior distributions
import aesara                    #Aesara is out tool for calculations involving tensors. PyMC will mostly work with this for us.
import pandas as pd              #Using pandas to read in CSV data files
import pickle                    #Pickle for saving data after completion of our model
import seaborn as sns            #We will use some of seaborn's distribution plotting tools
import pdb

Next up, let's load in our data. Importantly, Bayesian inference does not require additional preprocessing and runs in a similar manner to frequentist inference. In short, we process the data as necessary based on experimental design. Then we feed that directly into inference models.

In [ ]:
# Let's load some data!
#if __name__ == '__main__':       #This statement is to allow for parallel sampling in windows. Linux distributions don't require this. Doesn't hurt Linux to have this either.
print(f"Running on PyMC v{pm.__version__}")      #Tells us which version of PyMC we are running
AMDepthData = pd.read_csv("depthData.csv")      #Read in our data. Note this address may change based on where program files are downloaded.
    

This follows conventional PANDAS data loading (see PANDAS documentation for more details). Data is loaded as a dataframe object, where groups and covariates are columns of data. The if__name__ == '__main__' statement is a requirement for windows os that runs multiple threads. 

Lets take a second to explore our data

In [ ]:
display(AMDepthData)

As you can see, columns contain different features of the dataset. This is completely up to the design of the one acquiring data. All that is necessary is to know what feature we want to observe. Let's grab mean firing rates, since that is what we are interested in

In [ ]:
data1 = AMDepthData.loc[AMDepthData['Age'] == 2]                   #Let's consider young responses first.
data1.reset_index(drop=True, inplace = True)
AMDepthData = data1
"""
Now let's setup some meta data for our model analyses. We will set the number of burn in samples, which "primes" the markov chain Monte Carlo (MCMC) algorithm, and number of
samples to draw from the posterior. In general, less "well behaved" data will require more samples to get MCMC to converge to steady state. 
"""
numBurnIn = 2000
numSamples = 4000
#These next two lines of code setup the color scheme used for plotting. Does not affect model at all
color = '#87ceeb'
az.style.use("arviz-darkgrid")
Randomseed = 7

The first 3 lines are just PANDAS operations that grabs all data from all units that come from young animals (young class). Reset_index just organizes the data so that no zero'd rows are present from data that is in the aged class. 

We also begin to setup MCMC parameters numBurnIn and numSamples. numBurnIn starts an MCMC chain and runs it for a defined number of samples. This allows the chain to reach numerical stability before starting inference. The numSamples dictates how many draws from the posterior we want to make. This is dependent on the size and complexity of the data, which can be validated by observing the chains after convergence. These are mostly guided by heuristic looks at sampling chains to ensure proper sampling of the posterior. We will discuss that step in more detail later. For now, these parameters will work fine.

Now let's get mean firing rates and do some standard analytics before inference.

In [ ]:
modDepth = AMDepthData.ModDepth                     #This is our modulation depth vector
firingRate = AMDepthData['TotMean']                 #This is our mean firing rate. Note, data can be accessed in a pandas array using dot notation (data.subsetData) or
firingRateorig = firingRate                         #For comparing distributions
firingRate = np.log(firingRate+0.01)
                                                    #Index like data['subsetData']
firingRate = firingRate.astype(aesara.config.floatX) #Make sure response variable is in a tensor like structure for computaiton. This is the only time we need to directly invoke aesara
modDepth = np.asarray(modDepth)                     #Make sure Xs are not panda series, aesara is not a fan of those sometimes.
"""
Let's do some data visualization first. Always good to look at the data first. We will plot the scatter plot of the response variable and predictor variable
"""
plt.figure(1)
plt.scatter(modDepth,firingRate)
plt.show()
#Plot the distribution of response variable, firing rate
sns.displot(firingRate, kde=True)
plt.show()
sns.displot(firingRateorig, kde=True)
plt.show()

We begin by grabbing several key variables from our dataframe, namely SAM modulation depth and firing rates. It's always a good idea to visualize data before inference, regardless of if it is Bayesian or frequentist. Plotting the histogram and KDE distributions show that a log transform of firing rates results in a distribution that is more normal. We show in the paper that the log transformation with regression fits the data better (assuming the current model structure) than nonlog transformed data, so let's go with that. However, you can build a regression models with likelihoods that are not normal and fit the observed distribution and get the same results. However, normally distributed variables tend to be better for interpretation, both in inference and general data analyses writ large, so let's stick with that for now.

Now, let's build the regression model!

In [ ]:
prMean = np.mean(firingRate)
with pm.Model() as regression:                    #Define a model that we call regression
    a = pm.Normal('a', mu=prMean, sigma = 5)           #Normally distributed prior on a
    B = pm.Normal('B', mu=prMean, sigma = 5)           #Normally distributed prior on B
    eps = pm.HalfCauchy("eps", 5)                 #Model error prior, half Cauchy distributed with variance 5
    # Now we define our likelihood function, which for regression is our regression function
    reg = pm.Deterministic('reg', a + (B*modDepth))      #Deterministic is for non probabilistic data. This is a modification to help sampling, inference is still probabilistic
    likelihood = pm.Normal('Y',mu = reg, sigma = eps, observed = firingRate)    
    """
    And that's it! We've quickly, explicitly, and easily defined our model. We set prior distributions on a, B, and modelError, and defined a likelihood function of a normal linear regression
    with our observed data being our firingRate, which is the variable we are trying to predict. Easy eh?
    """ 


In this regression model, we set our prior distributions to be centered around observed data, (prMean) with large variance (sigma=5). This is to allow the data to "speak for itself" outside of any other prior information. Models are relatively easy to setup in PyMC, just requiring a declaration of the distribution and the distribution parameters (see PyMC API to find all distributions available). Our regression equation is firingRate = a + b*SAMDepth+error. In this case, we create a normal distribution for the a and b regression parameters, titled 'a' and 'b' respectively. Our error term is a HalfCauchy distribution, following suggestions by Kruscke and Gelman. The logic is as follows: errors are absolute value distances from regression line fits, hence a one-sided distribution. The half Cauchy distribution has rapdily falling tails, since small errors are more likely than large errors. However, the distribution has infinite support (tails never hit 0 finitely), so if a large error is present from the data, it will be represented in the posterior. 

We define the regression model 'reg' and place a normal distribution around the regression model, forming a likelihood distribution. Importantly, data is fed directly into the 'observed' variable of the model. Again, no other preprocessing is necessary; what is fed in is what is analyzed. 

And that's it! A full explicity declaration of the model. We find it helpful to fully define your models in code. We feel it helps researchers get good intuition about their data and inference on it.  However, packages such as Bambi can abstract this step to a single line of code (see Bambi regression model code).

Next, let's run our model!

In [ ]:
"""
Our last step is to run inference to get our posterior distribution which we do by MCMC sampling. In PyMC this is also easy.
"""
if __name__ == '__main__':    
    with regression:                 #Access our defined model
        trace = pm.sample(numSamples, tune=numBurnIn, target_accept=0.90,chains = 4)       #4 parallel, independent MCMC chains.
"""
Now we're sampling! We initialize the chain with numBurnIn samples, and then run for numSamples. Target_accept determines MCMC step size in order to get to a sampling acceptance rate. Generally,
higher probability of target_accept helps with difficult to sample posteriors. For now, 0.9 is fine.

You will see the sampling run in the command prompt. Once complete, we have our posterior and can now make inference!
"""

This step is the "inference button" step. Running this line of code completes inference and creates a posterior distribution. We are using parallel processing to run 4 chains in parallel (chains=4). We burn in an MCMC chain for numBurnIn and run it for numSamples. MCMC aims to estimate posterior distributions by sampling with Bayes rule. As such, it runs a rejection-sampling based No U-turn sampler. The full description of how this is done is out of scope for this tutorial, but effectively the sampler tries to generate samples from the posterior that fit into the posterior distribution with target)accept accuracy. An error will be thrown if this criterion is not met, indicating divergence from the posterior. Generally 90% results in well fitting posteriors to data. However, misbehaved data distributions, such as multi-peaked mixture of gaussians will require a more strict target_accept. Increasing target_accept increases computation time. 

Excellent, now that we have a posterior, let's visualize it! 

In [ ]:
intercept = trace.posterior["a"]                #Grab the posterior distribution of a
Slope = trace.posterior["B"]                    #Grab the posterior distribution of B
err = trace.posterior["eps"]                    #Grab the posterior distribution of model error


In [ ]:
Traces are literally a timeseries sample from MCMC sampling which have distributions following the posterior distribution. We can grab posterior distributions by simply quering the trace posteriors 
for each of our regression parameters defined above.

"""
Let's plot our posteriors!
"""
#az.plot_trace(trace, compact=True)
az.plot_posterior(trace, var_names=['a', 'B','eps'], point_estimate='mode',hdi_prob=0.95)
plt.show()
fig = plt.figure(figsize=(9, 6))
ax = fig.add_subplot(
111,
xlabel=r"Modulation Depth",
ylabel=r"Firing Rate",
title="Posterior predictive regression lines",
)
sc = ax.scatter(modDepth, firingRate)
#Plot posterior HDIs
az.plot_hdi(
    modDepth,
    trace.posterior.reg,
    color="k",
    hdi_prob=0.95,
    ax=ax,
    fill_kwargs={"alpha": 0.25},
    smooth=False,
)

This is the meat of Bayesian inference. Once we have our posteriors, we can easily do inference. We are primarily interested in HDI intervals from posterior distributions. 95% of our b parameter distribution lie outside of 0 with a credible region not including zero. This gives evidence to the fact that our regression slope parameter is significant.

The next question however is how well does our data fit the inference model? Let's do some checking.

In [ ]:
az.plot_energy(trace)
az.plot_trace(trace)
plt.show()
#pdb.set_trace()

We do some trace diagnostics first. The plot_trace command shows us the physical time-series trace of the data. We are looking for a "fuzzy catepillar" like appearance, as seen here. This shows that MCMC is effectively sampling the posterior, and not stuck in a region of sampling space. 

In [ ]:
"""
Now let's do some posterior predictive checks. PyMC has some nice functions that make this quite easy. We will also sample the posterior distribution for the
standard 16,000 samples, which for this posterior should be more than enough.
"""
with regression:
    ppcRegression = pm.sample_posterior_predictive(trace, random_seed=Randomseed)

#The above code envokes the regression mode, then uses the posterior from the trace, pulling synthetic samples to compare to observed. Random seed is set so that each run can be perfectly replicated
az.plot_bpv(ppcRegression, hdi_prob=0.95,kind='p_value')
#Bayes p-values, similar to frequentist,can be used to assess if posterior predictive is sufficiently close to observed density. Should be centered around 0.50.
az.plot_ppc(ppcRegression)
az.plot_trace(trace,var_names=['a', 'B','eps'])
plt.show()

This is a critical step in inference. We use posterior predictive checking to compare sample draws from the posterior to observed data distributions. We want to ensure that our statistical model
adequately fits our data. Plotting the regression model on the log transformed firing data shows good fits to observed data, suggesting that our model is correctly making inference. Rerunning this on nonlogtransformed data shows that the posterior does not represent our observed data, and thus any inference is not true to underlying data. 

A way of quantifying this is the Bayesian p-value, run by plot_bpv. This quantifies the disparity between observed and posterior distributions. Good fits have little disparity, with bpv distributions centered around 0.5. 